# PCA-eval : evaluating Principal Component Analysis (PCA) performance in high-dimensional data

I created this simple package to evaluate whether PCA sucessfully carries most of the data variance in single-cell genomics. It should work for any high-dimensional data. In a nutshell, this helps practitioners evaluate whether they can perform PCA for denoising and reduction of computational burden before proceeding with sophisticated non-linear methods.

Let's start to see how to use this package with some simple simulated data:

In [ ]:
from pcaeval.pca_eval import (generate_linear_data,
                              generate_uncorrelated_data,
                              evaluate_matrix,
                              plot_sing_vals_exp_var)

We'll simulate linearly correlated data and uncorrelated data to validate the analysis pipeline.

In [ ]:
linear_test = generate_linear_data(n=10000, d=2000, n_classes=10, redundancy=0.1, noise=0.1, seed=0)
uncorrelated_test = generate_uncorrelated_data(n=10000, d=2000, seed=0)

datasets = {'Linear data':linear_test, 'Uncorrelated data':uncorrelated_test}

Next, we'll evaluate these data matrices with our `evaluate_matrix()` function. It yields a dictionary of results.

In [ ]:
results_dict = {}
for i_data, name in enumerate(datasets):
    r_dict = evaluate_matrix(
        datasets[name], n_pcs=100, dimred_estimator=False,
     clustering_estimator=False, n_jobs=-1, verbose=True
     )
    results_dict[name] = r_dict

Finally, let's plot the covariance eigenspectrum (singular values and cumulative explained variance):

In [ ]:
plot_sing_vals_exp_var(results_dict, fontsize=10)

As one can see, PCs explain very little variance and fail to meaningfully represent the data when it is not linearly correlated.

An example of how this can be important is neighborhood graph construction: often machine-learning or bioinformatics practitioners learn nearest-neighbors graphs on top PCs instead of using the full data. However, the graph learned from the PCs is poorly correlated with the graph learned from the full data when features are not linearly correlated (i.e. data cannot be represented by hyperplanes of maximum covariance):

In [ ]:
print('Spearman R correlation between the k-nearest-neighbors graphs learned from the data and the Principal Components:' +
'\n Linear data: %f'%results_dict['Linear data']['graph_correlation'] + '\n Uncorrelated data: %f'%results_dict['Uncorrelated data']['graph_correlation'])

Thus, as we can see, building the k-nearest-neighbors graph from PCs when data features are not linearly correlated is often a bad idea. The same goes for other algorithms with locality-preserving characteristics, such as non-linear dimensional reduction.

Let's see how this goes for UMAP, a very popular dimensional reduction technique. Any method can be evaluated within `pca-eval`, but we currently expect dimensional reduction methods  to have a `.fit_transform(X)` method. If no method is provided, `pca-eval` will use t-SNE by default.

In [ ]:
# Install umap if you don't have it (c'mon, you should already have it, its fabulous)
#%pip install umap-learn

In [ ]:
from umap import UMAP

umap_estimator = UMAP(n_neighbors=10, min_dist=0.3, n_components=2, random_state=42)

results_dict = {}
for i_data, name in enumerate(datasets):
    r_dict = evaluate_matrix(
        datasets[name], n_pcs=100, dimred_estimator=umap_estimator,
     clustering_estimator=kmeans_estimator, n_jobs=-1, verbose=True
     )
    results_dict[name] = r_dict

In [ ]:
from pcaeval.pca_eval import print_dict_results

print_dict_results(results_dict)

Now let's see how this goes in single-cell genomics:

In [ ]:
import scanpy as sc
from pcaeval.pca_eval import evaluate_anndata, plot_sing_vals_exp_var
adata = sc.datasets.pbmc3k()

# Input: a filtered, unnormalized AnnData object
# If already normalized, set norm_log_hvg=False
res_dict, adata = evaluate_anndata(adata, norm_log_hvg=True)

In [ ]:
dict = {'PBMC3K':res_dict}
print_dict_results(dict)

In [ ]:
plot_sing_vals_exp_var(res_dict, dataset=False, fontsize=10)
# datset=False means that the dictionary is not a dictionary of dictionaries with multiple datasets, but
# a dictionary with results from a single dataset

As easily observed, single-cell genomics data should be considered as not having linearly correlated variables. Thus, PCA is unsuited for its preprocessing or analysis unless proven otherwise. 

----------
NOTICE: 
As a matter of fact, nearly all published literature in single-cell genomics uses PCA for processing, so one cannot rigorously trust any of it. As despairing as this might seem, I see this as an opportunity for the community to do better rather than just following tutorials from toolkits such as Seurat and Scanpy (which do PCA preprocessing by default). It is also an opportunity to work together as a community, since all of this data will need to be reanalysed so that artifacts and distortions can be identified.

One may wonder how the communitty allowed it to come to this. I believe the answer lies somewhere between the unwillingness to address mistakes in disregard of how malicious they might be for science and the inability to question whether what is done by all is actually correct.